# Python之建模灰色篇

* 多层模糊评价
* 模糊c均值聚类 （FCM聚类）
* 灰色预测 （GME）

![](./img/P9_1.png)

![](./img/P9_2.png)

![](./img/P9_3.png)

![](./img/P9_4.png)

![](./img/P9_5.png)

![](./img/P9_6.png)

![](./img/P9_7.png)

In [ ]:
import numpy as np

def frequency(matrix, p):
    A = np.zeros((matrix.shape[0]))
    for i in range(0, matrix.shape[0]):
        row = list(matrix[i,:])
        maximum = max(row)
        minimum = min(row)
        gap = (maximum - minimum) / p
        row.sort()
        group = []
        item = minimum
        while(item < maximum):
            group.append([item, item + gap])
            item = item + gap
            dataDuct = {}
            for k in range(0, len(group)):
                dataDict[str(k)] = 0
            for j in range(0, matrix.shape[1]):
                for k in range(0, len(group)):
                    if(matrix[k, j] >= group[k][0]):
                        dataDict[str(k)] = dataDict[str(k)] + 1
                    break
            index = int(max(dataDict, key=dataDict.get))
            mid = (group[index][0] + group[index][1]) / 2
            A[i] = mid
    A = A / sum(A[:])
    return A

![](./img/P9_8.png)

In [ ]:
def AHP(matrix):
    if isConsist(matrix):
        lam, x = np.linalg.eig(matrix)
        return x[0] / sum(x[0][:])
    else:
        print("一致性检验未通过")
        return None
    
def isConsist(matrix):
    n = np.shape(matrix)[0]
    a, b = np.linalg.eig(matrix)
    maxlam = a[0].real
    CI = (maxlam - n)/(n - 1)
    RI = [0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45]
    CR = CI/RI[n-1]
    if CR < 0.1:
        return True, CI, RI[n-1]
    else:
        return False, None, None

![](./img/P9_9.png)

In [ ]:
def appraise(criterionMatrix, targetMatrixs, relationMatrixs):
    R = np.zeros((criterionMatrix.shape[1], relationMatrixs[0].shape[1]))
    for index in range(0, len(targetMatrixs)):
        row = mul_mymin_operator(targetMatrixs[index], relationMatrixs[index])
        R[index] = row
    B = mul_mymin_operator(criterionMatrix, R)
    return B/sum(B[:])

def mul_mymin_operator(A, R):
    B = np.zeros((1, R.shape[1]))
    for column in range(0, R.shape[1]):
        list = []
        for row in range(0, R.shape[0]):
            list.append(A[row] * R[row, column])
        B[0, column] = mymin(list)
    return B

def mymin(list):
    for index in range(1, len(list)):
        if index == 1:
            temp = min(1, list[0]+list[1])
        else:
            temp = min(1, temp+list[index])
    return temp

![](./img/P9_10.png)

![](./img/P9_11.png)

![](./img/P9_12.png)

![](./img/P9_13.png)

![](./img/P9_14.png)

In [ ]:
import copy
import math
import random
import time

global MAX  # 用于初始化隶属度矩阵U
MAX = 10000.0
global Epsilon  # 结束条件
Epsilon = 0.0000001

def import_data_from_iris(file):
    data = []
    cluster_location = []
    with open(str(file), 'r') as f:
        for line in f:
            current = line.strip().split(",")
            current_dummy = []
            for j in range(0, len(current)-1):
                current_dummy.append(float(current[j]))
            j += 1
            if current[j] == "Iris-setosa\n":
                cluster_location.append(0)
            elif current[j] == "Iris-versicolor\n":
                cluster_location.append(1)
            else:
                cluster_location.append(2)
            data.append(current_dummy)
    return data

def randomize_data(data):
    order = list(range(0, len(data)))
    random.shuffle(order)
    new_data = [[] for i in range(0,len(data))]
    for index in range(0, len(data)):
        new_data[index] = data[order[index]]
    return new_data, order

def de_randomize_data(data, order):
    new_data = [[] for i in range(0, len(data))]
    for index in range(len(order)):
        new_data[order[index]] = data[index]
    return new_data

def print_matrix(list):
    for i in range(0, len(list)):
        print(list[i])
        
def initialize_U(data, cluster_number):
    global MAX
    U = []
    for i in range(0, len(data)):
        current = []
        rand_sum = 0.0
        for j in range(0, cluster_number):
            dummy = random.randint(1, int(MAX))
            current.append(dummy)
            rand_sum += dummy
        for j in range(0, cluster_number):
            current[j] = current[j] / rand_sum
        U.append(current)
    return U

def distance(point, center):
    if len(point) != len(center):
        return -1
    dummy = 0.0
    for i in range(0, len(point)):
        dummy += abs(point[i] - center[i]) **2
    return math.sqrt(dummy)

def end_condition(U, U_old):
    global Epsilon
    for i in range(0, len(U)):
        for j in range(0, len(U[0])):
            if abs(U[i][j]-U_old[i][j]) > Epsilon:
                return False
    return True

def normalize_U(U):
    for i in range(0, len(U)):
        maximum = max(U[i])
        for j in range(0, len(U[0])):
            if U[i][j] != maximum:
                U[i][j] = 0
            else:
                U[i][j] = 1
    return U

def fuzzy(data, cluster_number, m):
    U = initialize_U(data, cluster_number)
    while(True):
        U_old = copy.deepcopy(U)
        C = []
        for j in range(0, cluster_number):
            current_cluster_center = []
            for i in range(0, len(data[0])):
                dummy_sum_num = 0.0
                dummy_sum_dum = 0.0
                for k in range(0, len(data)):
                    dummy_sum_num += (U[k][j]**m)*data[k][i]
                    dummy_sum_dum += (U[k][j]**m)
                current_cluster_center.append(dummy_sum_num/dummy_sum_dum)
            C.append(current_cluster_center)
        distance_matrix=[]
        for i in range(0, len(data)):
            current = []
            for j in range(0, cluster_number):
                current.append(distance(data[i], C[j]))
            distance_matrix.append(current)
        for j in range(0, cluster_number):
            for i in range(0, len(data)):
                dummy = 0.0
                for k in range(0, cluster_number):
                    dummy += (distance_matrix[i][j]/distance_matrix[i][k])**(2/(m-1))
                    U[i][j] = 1/dummy
            if end_condition(U, U_old):
                print("结束聚类")
                break
        print("标准化U")
        U = normalize_U(U)
        return U
    
def checker_iris(final_location):
    right = 0.0
    for k in range(0, 3):
        checker = [0,0,0]
        for i in range(0, 50):
            for j in range(0, len(final_location[0])):
                if final_location[i+(50*k)][j] == 1:
                    checker[j] += 1
        right += max(checker)
    print("分类正确的个数是：", right)
    answer = right/150*100
    print("准确率："+str(answer)+"%")
    
data_iris = import_data_from_iris("iris.txt")
data_i, order = randomize_data(data_iris)
start = time.time()
final_location = fuzzy(data_i,3,2)
final_location = de_randomize_data(final_location, order)
print(checker_iris(final_location))
print("用时：{0}".format(time.time()-start))

![](./img/P9_15.png)

![](./img/P9_16.png)

![](./img/P9_17.png)

![](./img/P9_18.png)

![](./img/P9_19.png)

![](./img/P9_20.png)

In [12]:
import torch as th
import numpy as np

class GM():
    def __init__(self):
        self._is_gpu = False  # 判断是否可用GPU编程
    def fit(self, dt:list or np.ndarray):
        self._df:th.Tensor=th.from_numpy(np.array(dt,dtype=np.float32))
        if self._is_gpu:
            self._df.cuda()
        self._n:int = len(self._df)
        self._x,self._max_value = self._sigmod(self._df)
        z:th.Tensor = self._next_to_mean(th.cumsum(self._x,dim=0))
        self.coef:th.Tensor = self._coefficient(self._x, z)
        del z
        self._x0:th.Tensor = self._x[0]
        self._pre:th.Tensor = self._pred()
    def _sigmod(self, x:th.Tensor):
        _maxv:th.Tensor = th.max(x)
        return th.div(x, _maxv), _maxv
    def _next_to_mean(self, x_1:th.Tensor):
        z:th.Tensor = th.zeros(self._n-1)
        if self._is_gpu:
            z.cuda()
        for i in range(1, self._n):
            z[i-1]=0.5*x_1[i]+0.5*x_1[i-1]
        return z
    def _coefficient(self, x:th.Tensor, z:th.Tensor):
        B:th.Tensor = th.stack((-1*z, th.ones(self._n-1)),dim=1)
        Y:th.Tensor = th.tensor(x[1:], dtype=th.float32).reshape((-1,1))
        if self._is_gpu:
            B.cuda()
            Y.cuda()
        return th.matmul(th.matmul(th.inverse(th.matmul(B.t(),B)),B.t()),Y)
    def _pred(self, start:int=1, end:int=0):
        les:int = self._n+end
        result:th.Tensor = th.zeros(les)
        if self._is_gpu:
            result.cuda()
        result[0]=self._x0
        for i in range(start,les):
            result[i]=(self._x0 - (self.coef[1]/self.coef[0]))*(1-th.exp(self.coef[0]))*th.exp(-1*self.coef[0]*(i))
        del les
        return result
    def confidence(self):
        return round((th.sum(th.abs(th.div((self._x-self._pre),self._x)))/self._n).item(),4)
    def predict(self, m:int=1, decimals:int=4):
        y_pred:th.Tensor = th.mul(self._pre,self._max_value)
        y_pred = th.zeros(1)
        if m<0:
            return "预测个数需大于等于0"
        elif m>0:
            y_pred_:th.Tensor=self._pred(self._n,m)[-m:].mul(self._max_value)
        else:
            if self._is_gpu:
                return list(map(lambda _:round(_,decimals),y_pred.cpu().numpy().tolist()))
            else:
                return list(map(lambda _:round(_,decimals),y_pred.numpy().tolist()))
        result:th.Tensor = th.cat((y_pred,y_pred_),dim=0)
        del y_pred,y_pred_
        if self._is_gpu:
            return list(map(lambda _:round(_,decimals),result.cpu().numpy().tolist()))
        return list(map(lambda _:round(_,decimals),result.numpy().tolist()))
    
ls = np.arange(91,100,2)
print(type(ls))
gm = GM()
gm.fit(ls)
print(gm.confidence())
print(ls)
print(gm.predict(m=2))

<class 'numpy.ndarray'>
0.0002
[91 93 95 97 99]
[0.0, 101.1007, 103.2289]


D:\anaconda3\envs\swin\lib\site-packages\ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
